In [1]:
import json
import time
import requests
import re
from tqdm import tqdm_notebook as tqdm

In [2]:
headers = {
    "Content-Type": "application/json",
    'accept': 'application/json'
}

In [3]:
def json_load(name):
    with open(f'{name}', 'r', encoding = 'utf-8') as f:
        return json.load(f)
    
def json_save(name, item):
    with open(f'{name}', 'w', encoding = 'utf-8') as f:
        json.dump(item, f, ensure_ascii = False, indent = 2)

## Ask QAnswer QALD questions

In [4]:
qald = json_load("../processed_data/QALD/qald_train_wdt.json")

In [5]:
len(qald), qald[0]

(217,
 {'uid': '21',
  'question_text': 'Who wrote the book The pillars of the Earth?',
  'query_wikidata': 'PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { wd:Q1163227 wdt:P50 ?uri . }',
  'query_dbpedia': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?uri WHERE { res:The_Pillars_of_the_Earth dbo:author ?uri }',
  'answers': '',
  'question_text_ru': ['Кто написал книгу "Столпы Земли"?',
   'Кто автор книги "Столпы Земли"?'],
  'question_text_de': ['Wer schrieb die Säulen der Welt?']})

In [9]:
qanswer_responses = json_load("../processed_data/QALD/qanswer_train_responses.json")
ids = [q['uid'] for q in qanswer_responses]

for q in tqdm(qald):
    if q['uid'] not in ids:
        question = q['question_text']

        response = requests.get(
            "https://qanswer-core1.univ-st-etienne.fr/api/qa/full?question={0}&lang=en&kb=wikidata".format(question)
        ).json()['queries']

        qanswer_responses.append({
            'uid': q['uid'],
            'response': [{'query': r['query'], 'confidence': r['confidence']} for r in response]
        })
        time.sleep(1)
    
json_save("../processed_data/QALD/qanswer_train_responses.json", qanswer_responses)

/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


KeyboardInterrupt: 

## Execute QAnswer responses

`python scripts/qald_run_sparql_candidates_on_wikidata.py`

## Get true answers

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql")

In [13]:
test_new = list()
for q in tqdm(qald):
    time.sleep(1)
    try:
        sparql.setQuery(q['query_wikidata'])
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        answers = list()
        if "results" in results.keys():
            for result in results["results"]["bindings"]:
                answers.append(result)
        elif "boolean" in results.keys():
            answers = results["boolean"]
    except:
        time.sleep(1)
        try:
            sparql.setQuery(q['query_wikidata'])
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()

            answers = list()
            if "results" in results.keys():
                for result in results["results"]["bindings"]:
                    answers.append(result)
            elif "boolean" in results.keys():
                answers = results["boolean"]
        except:
            print(q['query_wikidata'])
            answers = None
    
    if answers:
        q['results_wikidata'] = answers    
        test_new.append(q)
    else:
        q['results_wikidata'] = None    
        test_new.append(q)

/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:845: RuntimeWarning: not detected query type for query 'information not available'
  warnings.warn("not detected query type for query '%s'" % query.replace("\n", " "), RuntimeWarning)
/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:852: RuntimeWarning: unknown query type 'None'
  warnings.warn("unknown query type '%s'" % r_queryType, RuntimeWarning)


information not available


/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:845: RuntimeWarning: not detected query type for query 'no data available'
  warnings.warn("not detected query type for query '%s'" % query.replace("\n", " "), RuntimeWarning)


no data available
information not available
no data available
no data available
no data available
no data available
no data available
no data available
no data available
no data available
no data available
SELECT DISTINCT ?uri WHERE { ?uri <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q6256>  . ?cave <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q35509> . { ?cave <http://www.wikidata.org/prop/direct/P17> ?uri . }  UNION { ?cave <http://www.wikidata.org/prop/direct/P131>/<http://www.wikidata.org/prop/direct/P17> ?uri . } } GROUP BY ?uri HAVING (COUNT(DISTINCT ?cave) > 10)


/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:845: RuntimeWarning: not detected query type for query 'no information available'
  warnings.warn("not detected query type for query '%s'" % query.replace("\n", " "), RuntimeWarning)


no information available



In [15]:
json_save("../processed_data/QALD/qald_train_wdt_answers.json", test_new)

## Get Labels from QALD

`python ../scripts/get_qald_labels_wikidata.py`

In [35]:
test_labels = json_load("../processed_data/QALD/qanswer_test_responses_labels.json")

## Mark if the question was correctly answered by QAnswer

In [24]:
def precision_recall(true: list, pred: list):
    """
    Take prediction for a given question 
    and check how many of them are good ones. 
    That number divided by number of predictions gives you the Precision.
    
    Recall how many true results are in predictions
    """
    if len(true) == 0 and len(pred) == 0:
        return 1, 1
    
    intersect = len(set(pred).intersection(set(true)))
    precision = intersect/len(pred) if len(pred) > 0 else 0
    recall = intersect/len(true) if len(true) > 0 else 0
    
    return precision, recall

def find_by_uid(uid, array):
    for q in array:
        if uid == q['uid']:
            return q
    assert False

In [25]:
test_new = json_load("../processed_data/QALD/qald_train_wdt_answers.json")
qanswer_results_new = json_load("../processed_data/QALD/qanswer_train_responses_extended.json")

In [27]:
for i in range(len(qanswer_results_new)):
    # get true answers
    true = list()
    test_new_i = find_by_uid(qanswer_results_new[i]['uid'], test_new)
    if type(test_new_i['results_wikidata']) == list:
        for k in range(len(test_new_i['results_wikidata'])):
            for value in list(test_new_i['results_wikidata'][k].values()):
                if value['value'] not in true:
                    true.append(value['value'])
    elif type(test_new_i['results_wikidata']) == bool:
        true = test_new_i['results_wikidata']
        
    # for each query candidate get predicted answers and mark if it was true
    for j in range(len(qanswer_results_new[i]['response'])):
        predicted = list()
        if type(test_new_i['results_wikidata']) != type(qanswer_results_new[i]['response'][j]['result']):
            qanswer_results_new[i]['response'][j]['is_true'] = False
        elif type(qanswer_results_new[i]['response'][j]['result']) == list:
            for k in range(len(qanswer_results_new[i]['response'][j]['result'])):
                for value in list(qanswer_results_new[i]['response'][j]['result'][k].values()):
                    if value['value'] not in predicted:
                        predicted.append(value['value'])

            prec, rec = precision_recall(true, predicted)
            if prec >= 0.5 and rec >= 0.5: # if candidate is correct
                qanswer_results_new[i]['response'][j]['is_true'] = True
            else:
                qanswer_results_new[i]['response'][j]['is_true'] = False
        elif type(qanswer_results_new[i]['response'][j]['result']) == bool:
            if qanswer_results_new[i]['response'][j]['result'] == true:
                qanswer_results_new[i]['response'][j]['is_true'] = True
            else:
                qanswer_results_new[i]['response'][j]['is_true'] = False

In [28]:
json_save("../processed_data/QALD/qanswer_train_responses_extended.json", qanswer_results_new)

In [29]:
# load test results to TEST it
qanswer_results_new = json_load("../processed_data/QALD/qanswer_test_responses_extended.json")

## Evaluate the classifier

In [30]:
def precision_at_k(data, k=1):
    """
    How many relevant items are present in the top-k recommendations of the system
    """
    assert k > 0
    prec = list()
    for q in data:
        cnt = 0
        for i in range(len(q['response'])):
            if i + 1 <= k: # take first k responses
                if q['response'][i]['is_true']:
                    cnt +=1
        prec.append(cnt/k)
        
    return sum(prec)/len(prec)

In [31]:
precision_at_k(qanswer_results_new, 1), precision_at_k(qanswer_results_new, 5) # for initial set

(0.2882882882882883, 0.12612612612612611)

In [33]:
qald_test = json_load("../processed_data/QALD/qald_test_wdt.json")

In [53]:
qanswer_results_new[0]

{'uid': '201',
 'response': [{'query': 'SELECT DISTINCT ?o1 ?o2 WHERE { \t <http://www.wikidata.org/entity/Q331630>  <http://www.wikidata.org/prop/direct/P176>  ?o1 . \t ?o1  <http://www.wikidata.org/prop/direct/P571>  ?o2 .  }  LIMIT 1000',
   'confidence': 0.0,
   'result': [{'o1': {'type': 'uri',
      'value': 'http://www.wikidata.org/entity/Q948831'},
     'o2': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
      'type': 'literal',
      'value': '1992-01-01T00:00:00Z'}},
    {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q948831'},
     'o2': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
      'type': 'literal',
      'value': '1842-01-01T00:00:00Z'}}],
   'is_true': True},
  {'query': 'SELECT DISTINCT ?o2 WHERE { \t <http://www.wikidata.org/entity/Q331630>  <http://www.wikidata.org/prop/direct/P176>  ?o1 . \t ?o1  <http://www.wikidata.org/prop/direct/P571>  ?o2 .  }  LIMIT 1000',
   'confidence': 0.0,
   'result': [{'o2': {'datatype': 'ht

In [49]:
# filter answer candidates and create new dataset for evaluation
qanswer_results_filtered = list()

for i in tqdm(range(len(qanswer_results_new))):
    question_text = qald_test[i]['question_text']
    batch = list()
    for j in range(len(test_labels[i]['responses'])):
        answer_text = ' '.join(t for t in test_labels[i]['responses'][j])
        batch.append([question_text, answer_text])
    
    data = json.dumps(batch, ensure_ascii=False)
    data = data.encode('ascii', 'ignore').strip()

    json_response = requests.post('http://webengineering.ins.hs-anhalt.de:41003/predict',
                                  data=data,
                                  headers=headers)

    preds = json_response.json()['predictions'] # get predictions for the q-a tuples set
    answers = list()
    for j in range(len(preds)):
        if preds[j]:
            answers.append({'is_true': qanswer_results_new[i]['response'][j]['is_true']})
    qanswer_results_filtered.append({'response': answers})
    
# qanswer_results_filtered = [{'response': q} for q in qanswer_results_filtered]

/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [50]:
precision_at_k(qanswer_results_filtered, k=1), precision_at_k(qanswer_results_filtered, k=5)

(0.2072072072072072, 0.09729729729729727)